In [ ]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import word_tokenize


df = pd.read_csv('/Users/salil/Downloads/spam.csv',encoding='latin-1')

df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
df = df.rename(columns = {'v1':'spam','v2':'message'})
df['spam'] = df["spam"].map({'spam':1,'ham':0})
df['length'] = df['message'].apply(len)

text_message = df['message'].copy()
ham_words = ''
spam_words = ''
df_spam = df[df.spam == 1]
df_ham = df[df.spam ==0]


for val in df_spam.message:
    text = val.lower()
    tokens = word_tokenize(text)
    for words in tokens:
        spam_words = spam_words + words + ' '
        
for val in df_ham.message:
    text = val.lower()
    tokens = word_tokenize(text)
    for words in tokens:
        ham_words = ham_words + words + ' '
# Generate a word cloud image
spam_wordcloud = WordCloud(width=600, height=400).generate(spam_words)
ham_wordcloud = WordCloud(width=600, height=400).generate(ham_words)
#Spam Word cloud
plt.figure( figsize=(10,8), facecolor='k')
plt.imshow(spam_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


#Ham Word cloud
plt.figure( figsize=(10,8), facecolor='k')
plt.imshow(ham_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


In [ ]:
def text_preprocess(text):
    text = [word.lower() for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

text_process = text_message.apply(text_preprocess)
print text_process

def stemmer(text):
    text = text.split()
    words = ""
    porter = PorterStemmer()
    for i in text:
        #unicode(i, errors='replace')
        words += (porter.stem(i))+" "
    return words
text_stem = text_process.apply(stemmer)
print text_stem

def lemmer(text):
    text = text.split()
    words = ""
    lem = WordNetLemmatizer()
    for i in text:
        #unicode(i, errors='replace')
        words += (lem.lemmatize(i))+" "
    return words
text_lem = text_stem.apply(lemmer)
print text_lem

In [ ]:
vectorizer = TfidfVectorizer("english")
features = vectorizer.fit_transform(text_lem)
print features

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(features, df['spam'], test_size=0.3, random_state=111)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
nbc = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)

In [ ]:
models = {'SVC' : svc,'KNN' : knc, 'NB': nbc, 'DT': dtc, 'LR': lrc, 'RF': rfc, 'AdaBoost': abc}

def train_classifier(model, feature_train, labels_train):    
    model.fit(feature_train, labels_train)
def predict_labels(model, features):
    return (model.predict(features))

In [ ]:
pred_scores = []
for k,v in models.items():
    train_classifier(v, features_train, labels_train)
    pred = predict_labels(v,features_test)
    pred_scores.append((k, [accuracy_score(labels_test,pred)]))

In [ ]:
print pred_scores

In [ ]:
df_score = pd.DataFrame(pred_scores, columns=['Classifier','Score'])
print df_score